# Custom Logging
This jupyter notebook is an example of how to implement custom logging in Microsoft Fabric using an Eventhouse. You can find my blog on this topic at [kevinoftech.com](https://www.kevinoftech.com/Blog/Post/2026-01-fabric-custom-logging).

## Import Python Libraries

In [ ]:
from datetime import datetime

## Setup Variables
#### logs
This is going to store all the logs in our notebook that will be written to the Eventhouse.
#### notebook_name
Using notebookutils.runtime.context we can get the name of the current notebook. This is helpful so that we don't have to update this variable in every new notesbook we copy this to. 
#### workspace_name
Using notebookutils.runtime.context we can get the name of the current workspace. This is helpful so that we don't have to update this variable in every new notesbook we copy this to. 

In [ ]:
logs = []
notebook_name = notebookutils.runtime.context.get("currentNotebookName")
workspace_name = notebookutils.runtime.context.get("currentWorkspaceName")

## Log Function
We will use this function to add logs to the log variable we set above. 

Notice that the log_trace parameter is optional. 

In [ ]:
def log(log_level, log_message, log_trace=None):
    logs.append({
        "log_datetime": datetime.now(), 
        "workspace_name": workspace_name,
        "notebook_name": notebook_name,
        "log_level": log_level,
        "log_message": log_message,
        "log_trace": log_trace
    })

## Write to Eventhouse Function
This function is called at the end of the notebook to actually write the logs to the Eventhouse. 

#### Note 1
Make sure all the variables in this code block are set correctly.
- database
- table
- eventhouse_uri
- columns_in_order

#### Note 2
Notice the array "columns_in_order". This has to be set for when we call our spark.createDataFrame() we can set the order of the columns in the dataframe. If we do not do this the data will be sent to the Eventhouse in the wrong columns.

In [ ]:
def write_to_eventhouse(logs):
    database = "Logging"
    table = "NotebookLogging"
    eventhouse_uri = "https://<your-kusto-cluster>.kusto.fabric.microsoft.com"
        
    columns_in_order = [
        "log_datetime", 
        "workspace_name",
        "notebook_name", 
        "log_level", 
        "log_message", 
        "log_trace"
    ]

    try:
        df = spark.createDataFrame(logs).select(columns_in_order) # .select() puts the columns in the correct order
        
        df.write.format("com.microsoft.kusto.spark.synapse.datasource").\
        option("kustoCluster", eventhouse_uri).\
        option("kustoDatabase", database).\
        option("kustoTable", table).\
        option("accessToken", mssparkutils.credentials.getToken(eventhouse_uri)).\
        option("tableCreateOptions", "CreateIfNotExist").mode("Append").save()
    except Exception as e:
        print(e)

## Test Creating Logs
The log() function has an optional parameter called "log_trace".
#### Test 1: No Trace
In this example, we leave the "log_trace" parameter blank since there is nothing to trace. 
#### Test 2: With trace
In this example, we trigger an error and capture it so we can see that a trace would look like in the logs. It should look something like this

```
Traceback (most recent call last):\n  File \"/tmp/ipykernel_5632/1710815062.py\", line 8, in <module>\n    1 / 0  # This raises a ZeroDivisionError\n    ~~^~~\nZeroDivisionError: division by zero\n
```


In [ ]:
# Test 1: No Trace
log("Warning", "Test message 1")

# Test 2: With trace
import traceback

try:
    1 / 0  # This raises a ZeroDivisionError
except Exception as e:
    log("Warning", "Test message 2", traceback.format_exc())

## Write the Logs to the Eventhouse
This will write the content of logs array to the Eventhouse.

In [ ]:
write_to_eventhouse(logs)